In [1]:
from llama_cpp import Llama
from rich import print
from langchain_core.utils.function_calling import convert_to_openai_function
from llama_cpp.llama_tokenizer import LlamaHFTokenizer

from agents.specs import ChatCompletion
from agents.tool_executor import ToolRegistry

In [2]:
llm = Llama.from_pretrained(
    repo_id="meetkai/functionary-small-v2.2-GGUF",
    filename="functionary-small-v2.2.q4_0.gguf",
    chat_format="functionary-v2",
    tokenizer=LlamaHFTokenizer.from_pretrained("meetkai/functionary-small-v2.2-GGUF"),
    n_gpu_layers=-1,
    verbose=False,
)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [12]:
from agents.tools import get_current_weather

oai_tool = convert_to_openai_function(get_current_weather)
print(oai_tool)

registry = ToolRegistry()
registry.register_tool(get_current_weather)

{
    'name': 'get_current_weather',
    'description': 'get_current_weather(city: str) -> dict - Get the current weather for a given city.\n\n    
Args:\n      city (str): The city to fetch weather for.\n\n    Returns:\n      str: current weather condition, or 
None if an error occurs.',
    'parameters': {'type': 'object', 'properties': {'city': {'type': 'string'}}, 'required': ['city']}
}

In [15]:
output = llm.create_chat_completion(
    messages=[{"role": "user", "content": "How is the weather in SF today?"}],
    tools=[{"type": "function", "function": oai_tool}],
)

output = ChatCompletion(**output)

In [16]:
tool_response = registry.call_tool(c)
print(tool_response)

[
    {
        'tool_call_id': 'call_rHAK9zzYnIYYoU0zS2McvPeE',
        'role': 'tool',
        'name': 'get_current_weather',
        'content': [
            {
                'FeelsLikeC': '13',
                'FeelsLikeF': '56',
                'cloudcover': '75',
                'humidity': '94',
                'localObsDateTime': '2024-05-06 08:35 PM',
                'observation_time': '07:35 PM',
                'precipInches': '0.0',
                'precipMM': '0.3',
                'pressure': '1011',
                'pressureInches': '30',
                'temp_C': '14',
                'temp_F': '57',
                'uvIndex': '3',
                'visibility': '7',
                'visibilityMiles': '4',
                'weatherCode': '299',
                'weatherDesc': [{'value': 'Mist, rain'}],
                'weatherIconUrl': [{'value': ''}],
                'winddir16Point': 'NE',
                'winddirDegree': '40',
                'windspeedKmph': '9',
                'windspeedMiles': '6'
            }
        ]
    }
]